In [1]:
 %%capture
!pip install transformers -q
!pip install sentencepiece -q

In [2]:
import multiprocessing as mp
import random
import os
import os.path as osp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split
from transformers import ViTFeatureExtractor, VisionEncoderDecoderModel, GPT2TokenizerFast
from torch.utils.data import DataLoader

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import time
start = time.time()
!cp -r '/content/drive/MyDrive/Sampled_Images' '/content/'

In [ ]:
!cd '/content/drive/MyDrive/Sampled_Images' && ls | wc -l

^C


In [4]:
# load a fine-tuned image captioning model and corresponding tokenizer and feature extractor
model = VisionEncoderDecoderModel.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
tokenizer = GPT2TokenizerFast.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
feature_extractor = ViTFeatureExtractor.from_pretrained("nlpconnect/vit-gpt2-image-captioning")


Downloading:   0%|          | 0.00/4.61k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/982M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/241 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/798k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/120 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/228 [00:00<?, ?B/s]

In [5]:
PATH_DIR = '/content/drive/MyDrive/Sampled_Images/'

In [6]:
total_images = 11614

In [7]:
from os import listdir
 
# get the path/directory
folder_dir = PATH_DIR

captioned_count = 0
duplicate_image_id = []
caption_dict = {}


In [13]:
if torch.cuda.is_available():
  device = torch.device("cuda")
else:
  device = torch.device("cpu")

model = model.to(device)

In [14]:
device

device(type='cuda')

In [15]:
for image in tqdm(os.listdir(folder_dir)):
  if image in caption_dict.keys():
    continue

  try:
    image_path = PATH_DIR + image
    im = Image.open(image_path) 
    im = im.resize((224,224) ,resample = Image.LANCZOS)

    pixel_values = feature_extractor(im, return_tensors="pt").pixel_values.to(device)

    # autoregressively generate caption (uses greedy decoding by default)
    generated_ids = model.generate(pixel_values)
    generated_text = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

  except:
    continue

  captioned_count += 1
  if image not in caption_dict.keys():
    caption_dict[image] = generated_text
  else:
    duplicate_image_id.append(image)

  # if captioned_count % 100 == 0:
  #   print(f"Finished captioning batch of 100. Progres: {(captioned_count/total_images * 1.0) * 100}")

  0%|          | 0/11614 [00:00<?, ?it/s]

/usr/local/lib/python3.8/dist-packages/transformers/generation/utils.py:1387: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 20 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [16]:
import csv

header = ['image_id', 'caption']

with open('image_captions.csv', 'a', encoding = 'UTF8') as fp:
  writer = csv.writer(fp)

  writer.writerow(header)

  for image in caption_dict.keys():
    row = [image, caption_dict[image]]
    writer.writerow(row)

In [17]:
from google.colab import files
files.download('image_captions.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>